# 3 Pointers Made against game_totals.csv

### Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [2]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA\backend\analysis\3p
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [3]:
df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)

### Basic exploration

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40068 entries, 0 to 40067
Data columns (total 22 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     40068 non-null  object 
 1   visitor  40068 non-null  object 
 2   home     40068 non-null  object 
 3   team     40068 non-null  int64  
 4   fg       40054 non-null  float64
 5   fga      40054 non-null  float64
 6   fg_perc  40054 non-null  float64
 7   3p       40054 non-null  float64
 8   3pa      40054 non-null  float64
 9   3p_perc  40054 non-null  float64
 10  ft       40054 non-null  float64
 11  fta      40054 non-null  float64
 12  ft_perc  40054 non-null  float64
 13  orb      40054 non-null  float64
 14  drb      40054 non-null  float64
 15  trb      40054 non-null  float64
 16  ast      40054 non-null  float64
 17  stl      40054 non-null  float64
 18  blk      40054 non-null  float64
 19  tov      40054 non-null  float64
 20  pf       40054 non-null  float64
 21  pts      400

In [5]:
df.sample(5)

,date,visitor,home,team,fg,fga,fg_perc,3p,3pa,3p_perc,ft,fta,ft_perc,orb,drb,trb,ast,stl,blk,tov,pf,pts
17809,"Thu, May 2, 2013",Brooklyn Nets,Chicago Bulls,1,36.0,89.0,0.404,7.0,18.0,0.389,13.0,18.0,0.722,15.0,31.0,46.0,23.0,4.0,7.0,16.0,26.0,92.0
12434,"Fri, Mar 11, 2011",Los Angeles Clippers,New Jersey Nets,0,41.0,91.0,0.451,4.0,17.0,0.235,12.0,20.0,0.600,11.0,34.0,45.0,22.0,13.0,5.0,15.0,19.0,98.0
27972,"Sun, Mar 26, 2017",Chicago Bulls,Milwaukee Bucks,0,46.0,85.0,0.541,10.0,21.0,0.476,7.0,10.0,0.700,10.0,39.0,49.0,33.0,4.0,2.0,14.0,16.0,109.0
29415,"Tue, Dec 26, 2017",Brooklyn Nets,San Antonio Spurs,1,41.0,80.0,0.513,7.0,16.0,0.438,20.0,24.0,0.833,5.0,38.0,43.0,18.0,7.0,5.0,12.0,14.0,109.0
32721,"Mon, Feb 11, 2019",Milwaukee Bucks,Chicago Bulls,1,37.0,96.0,0.385,9.0,28.0,0.321,16.0,18.0,0.889,9.0,36.0,45.0,24.0,8.0,2.0,13.0,22.0,99.0


In [6]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])

# Conver 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])

# Statistics
stats = ['fg', 'fga', 'fg_perc', '3p', '3pa', '3p_perc', 'ft', 'fta', 'ft_perc', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40068 entries, 0 to 40067
Data columns (total 22 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     40068 non-null  datetime64[ns]
 1   visitor  40068 non-null  object        
 2   home     40068 non-null  object        
 3   team     40068 non-null  object        
 4   fg       40054 non-null  float64       
 5   fga      40054 non-null  float64       
 6   fg_perc  40054 non-null  float64       
 7   3p       40054 non-null  float64       
 8   3pa      40054 non-null  float64       
 9   3p_perc  40054 non-null  float64       
 10  ft       40054 non-null  float64       
 11  fta      40054 non-null  float64       
 12  ft_perc  40054 non-null  float64       
 13  orb      40054 non-null  float64       
 14  drb      40054 non-null  float64       
 15  trb      40054 non-null  float64       
 16  ast      40054 non-null  float64       
 17  stl      40054 non-null  float6

# Dataframe of team's last 15 performances

In [7]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = df[df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

df['dates'] = df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
df['date_1'], df['date_2'] = df['dates'].apply(lambda x: x[0]), df['dates'].apply(lambda x: x[1])
df['date_3'], df['date_4'] = df['dates'].apply(lambda x: x[2]), df['dates'].apply(lambda x: x[3])
df['date_5'], df['date_6'] = df['dates'].apply(lambda x: x[4]), df['dates'].apply(lambda x: x[5])
df['date_7'], df['date_8'] = df['dates'].apply(lambda x: x[6]), df['dates'].apply(lambda x: x[7])
df['date_9'], df['date_10'] = df['dates'].apply(lambda x: x[8]), df['dates'].apply(lambda x: x[9])
df['date_11'], df['date_12'] = df['dates'].apply(lambda x: x[10]), df['dates'].apply(lambda x: x[11])
df['date_13'], df['date_14'] = df['dates'].apply(lambda x: x[12]), df['dates'].apply(lambda x: x[13])
df['date_15'] = df['dates'].apply(lambda x: x[14])

In [8]:
# X and y column names to merge on
y_cols = df.columns
x_cols = ['date', 'team'] + stats

last_15_games = df[y_cols]
last_15_games['target'] = last_15_games['3p']
X = df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

In [9]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

In [10]:
# Calculate perc difference
def perc_diff(value, mean):
    return (value - mean) / mean

### Last 15 Performances (Unweighted)

In [11]:
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))]

last_15_games_unweighted = last_15_games[cols].copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15_games_unweighted[stat] = 0
    for date in dates:
        last_15_games_unweighted[stat] = last_15_games_unweighted[stat] + last_15_games_unweighted[stat + date]
    
    last_15_games_unweighted[stat] = last_15_games_unweighted[stat] / len(dates)
    
# Calculate standard deviation for each stat over a team's performance
for stat in stats:
    last_15_games_unweighted[stat + '_std'] = 0
    for date in dates:
        last_15_games_unweighted[stat + '_std'] = last_15_games_unweighted[stat + '_std'] + \
                                                    ((last_15_games_unweighted[stat + date] - last_15_games_unweighted[stat]) ** 2)
    
    last_15_games_unweighted[stat + '_std'] = last_15_games_unweighted[stat + '_std'] / len(dates)
    last_15_games_unweighted[stat + '_std'] = last_15_games_unweighted[stat + '_std'] ** .5

# Feature engineer trends
for stat in stats:
    last_15_games_unweighted[stat + '_trend'] = 0
    for date in dates[:10]:
        last_15_games_unweighted[stat + '_trend'] = last_15_games_unweighted[stat + '_trend'] + \
                                        z_score(last_15_games_unweighted[stat + date], last_15_games_unweighted[stat], last_15_games_unweighted[stat + '_std']).fillna(0)
    
    last_15_games_unweighted[stat + '_trend'] = last_15_games_unweighted[stat + '_trend'] / len(dates[:10])

last_15_games_unweighted = last_15_games_unweighted.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_15_game_cols = [col 
                    for col in last_15_games_unweighted.columns
                    if (col[0] == 'target' and col[1] == 'sum') or \
                       (col[0] in stats and col[1] == 'sum' and '_perc' not in col[0]) or \
                       (col[0] in stats and col[1] == 'mean' and '_perc' in col[0]) or \
                       ('_trend' in col[0] and col[1] == 'sum')]

last_15_games_unweighted = last_15_games_unweighted[last_15_game_cols].dropna(axis=0)
last_15_games_unweighted.columns = [col[0] for col in last_15_games_unweighted.columns]
last_15_games_unweighted.head()

target         fg  \
date       visitor         home                                        
2006-11-28 Indiana Pacers  Portland Trail Blazers    13.0  33.933333   
           New York Knicks Chicago Bulls              9.0  35.466667   
2006-11-29 Indiana Pacers  Golden State Warriors     18.0  74.133333   
           New York Knicks Cleveland Cavaliers       13.0  35.000000   
           Orlando Magic   Seattle SuperSonics        8.0  72.733333   

                                                          fga   fg_perc  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers   73.666667  0.460733   
           New York Knicks Chicago Bulls            78.933333  0.451000   
2006-11-29 Indiana Pacers  Golden State Warriors   162.800000  0.455100   
           New York Knicks Cleveland Cavaliers      77.200000  0.454800   
           Orlando Magic   Seattle SuperSonics     155.933333  0.469033   

                                                          3p        3pa  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers   4.733333  14.733333   
           New York Knicks Chicago Bulls            5.066667  15.400000   
2006-11-29 Indiana Pacers  Golden State Warriors   14.533333  40.800000   
           New York Knicks Cleveland Cavaliers      4.533333  14.533333   
           Orlando Magic   Seattle SuperSonics     11.000000  30.800000   

                                                    3p_perc         ft  \
date       visitor         home                                          
2006-11-28 Indiana Pacers  Portland Trail Blazers  0.326267  20.800000   
           New York Knicks Chicago Bulls           0.309200  22.933333   
2006-11-29 Indiana Pacers  Golden State Warriors   0.358433  38.400000   
           New York Knicks Cleveland Cavaliers     0.293667  22.200000   
           Orlando Magic   Seattle SuperSonics     0.355267  40.733333   

                                                         fta   ft_perc  \
date       visitor         home                                          
2006-11-28 Indiana Pacers  Portland Trail Blazers  26.866667  0.780467   
           New York Knicks Chicago Bulls           30.466667  0.754267   
2006-11-29 Indiana Pacers  Golden State Warriors   52.933333  0.727767   
           New York Knicks Cleveland Cavaliers     30.066667  0.740200   
           Orlando Magic   Seattle SuperSonics     54.800000  0.748500   

                                                         orb        drb  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers   9.466667  26.400000   
           New York Knicks Chicago Bulls           12.333333  31.066667   
2006-11-29 Indiana Pacers  Golden State Warriors   23.466667  58.466667   
           New York Knicks Cleveland Cavaliers     11.600000  30.466667   
           Orlando Magic   Seattle SuperSonics     23.666667  57.133333   

                                                         trb        ast  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers  35.866667  17.266667   
           New York Knicks Chicago Bulls           43.400000  17.533333   
2006-11-29 Indiana Pacers  Golden State Warriors   81.933333  46.600000   
           New York Knicks Cleveland Cavaliers     42.066667  17.333333   
           Orlando Magic   Seattle SuperSonics     80.800000  37.466667   

                                                         stl        blk  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers   6.466667   4.133333   
           New York Knicks Chicago Bulls            6.533333   3.266667   
2006-11-29 Indiana Pacers  Golden State Warriors   16.000000  13.333333   
           New York Knicks Cl

## Correlations

In [12]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15_games_unweighted:
    corr_p = pearsonr(last_15_games_unweighted['target'], last_15_games_unweighted[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print correlations
corr_df = corr_df[corr_df['p-value'] < .05].drop(['p-value'], axis=1).sort_values(['corr'], axis=0, ascending=False)
corr_df

,corr,stat
0,1.00,target
5,0.74,3pa
4,0.73,3p
18,0.55,pts
11,0.50,drb
2,0.48,fga
1,0.47,fg
13,0.43,ast
12,0.34,trb
9,0.17,ft_perc


## Save dataframe with significantly correlated stats

In [13]:
stats = corr_df[corr_df['corr'].abs() >= .4]['stat']
df = last_15_games_unweighted[stats]

df.to_csv('backend/data/inputs/3p/game_totals.csv')